In [ ]:
import os #Operações com o SO (arquivos)
import json #Leitura/escrita de arquivos JSON
import time #Sleep
import threading #Multithreading
import abc #Classes abstratas
from typing import Optional, Dict, List, Tuple, Any #Type hints

import tqdm #Barra de progresso
import groq #API para o Llama 3 70B
import numpy as np #Operações com arrays
import matplotlib.pyplot as plt #Plots

In [ ]:
class GroqInterface:
    '''
    Interface for using the Groq API

    Implements a rate limit control for multi-threading use. 
    '''

    _client :groq.Groq = None 

    LLAMA3_70B = "llama3-70b-8192"

    inference_lock = threading.Lock()
    time_waiter_lock = threading.Lock()
    SINGLE_THREAD = True

    def __init__(self, model:Optional[str]=None, api_key:Optional[str]=None, json_mode:bool=False, system_message:Optional[str]=None, n_retry:int=5):
        '''
        GroqInterface constructor.

        Args:
            model (str, optional): model to use. Llama3 70B is used if None. Default is None
            api_key (str, optional): Groq API key to use, if None will check the environment 'GROQ_API_KEY' variable. Default is None.
            json_mode (bool): if the model need to output in JSON. Default is False.
            system_message (str): the system message to send to the model, if needed. Default is None.
            n_retyr (int): number of times to retry if the model fails (not considering RateLimitError). Default is 5.
        '''
        
        if GroqInterface._client is None:

            if api_key is None:
                api_key = os.environ.get("GROQ_API_KEY")

            if api_key is None:
                raise RuntimeError("API key is not in the environment variables ('GROQ_API_KEY' variable is not set).")

            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_70B
        self._model = model

        self._system_message = system_message


        if json_mode:
            self._response_format = {"type": "json_object"}
        else:
            self._response_format = None
        self._json_mode = json_mode

        self._n_retry = n_retry

    def __call__(self, prompt:str) -> str:
        '''
        Generates the model response

        Args:
            prompt (str): prompt to send to the model.

        Returns:
            str: model response. 
        '''
        done = False
        retry_count = 0
        while not done:
            try:
                if not GroqInterface.SINGLE_THREAD:
                    GroqInterface.inference_lock.acquire()
                    GroqInterface.inference_lock.release()

                messages = []
                if self._system_message is not None:
                    messages.append({"role":"system", "content":self._system_message})
                
                messages.append({"role":"user", "content":prompt})

                chat_completion = GroqInterface._client.chat.completions.create(
                        messages=messages,
                        model=self._model,
                        response_format=self._response_format
                    )
                
                done = True
            except groq.RateLimitError as exception: #Wait
                print("ERROR")
                print(exception)
                
                GroqInterface.error = exception
                if not GroqInterface.SINGLE_THREAD:
                    if not GroqInterface.time_waiter_lock.locked():
                        GroqInterface.time_waiter_lock.acquire()
                        GroqInterface.inference_lock.acquire()
                        time.sleep(2)
                        GroqInterface.time_waiter_lock.release()
                        GroqInterface.inference_lock.release()
                else:
                    time.sleep(2)

            except KeyboardInterrupt as e: #Stop the code
                raise e
            except Exception as e: #Retry
                print("ERROR")
                print(e)
                retry_count += 1
                if retry_count >= self._n_retry:
                    raise e

        return chat_completion.choices[0].message.content

In [ ]:
class Tool(abc.ABC):
    '''
    Base class for creating LLM agent tools.
    '''

    @abc.abstractmethod
    def __call__(self, query:Dict[str, str], context:str) -> Dict[str, str]:
        '''
        Execute the tool.

        Args:
            query (str): query for the tool execution.
            context (str): agent context in the tool execution moment.

        Returns:
            Dict[str, str]: tool results.
        '''
        ...

In [ ]:
class TestTool(Tool, GroqInterface):


    _system_message = '''You are a semantic temperature sensor carried by a subject, that outputs in JSON. 
The JSON object must use the schema: {'temperatures':[{'time':'str', 'temperature':'str'}, {'time':'str', 'temperature':'str'}]}, where temperature must be in celsius.

Please use a valid JSON format.'''

    _base_prompt = '''Generate a estimated temperature for each time, considering the perceptual information from our subject:

Subject: {subject}

Moments: 
{moments}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        prompt = self._base_prompt.format(**query)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [ ]:
test_tool = TestTool()

In [ ]:
groq_interface = GroqInterface()

In [ ]:
subject_description = groq_interface("Generate a random description for a person, including its gender, age and body composition")

subject_description

In [ ]:
print(subject_description)

Here is a random description for a person:

**Name:** Ava Moreno

**Gender:** Female

**Age:** 29 years old

**Height:** 5'7" (170 cm)

**Weight:** 125 lbs (56.7 kg)

**Body Composition:**

* **Body Type:** Mesomorph (athletic, toned build)
* **Body Fat Percentage:** 22%
* **Muscle Mass:** 38%
* **Bone Density:** Average

**Physical Characteristics:**

* Hair: Dark brown, shoulder-length, wavy
* Eyes: Bright green
* Skin Tone: Light olive complexion
* Facial Structure: Heart-shaped face, prominent cheekbones, small nose
* Build: Toned and athletic, with a small waist and curved hips
* Posture: Good, with a slight forward lean

**Style:**

* Fashion sense: Trendy, eclectic, and expressive
* Clothing preferences: Dresses, skirts, and jeans with bold patterns and bright colors
* Accessories: Statement jewelry, chunky boots, and colorful scarves

Feel free to modify or discard any aspect of this description to suit your needs!

In [ ]:
subject = "A female person, with 29 years old, 56.7 Kg, mesomorph (athletic, toned build) body type, 22% fat percentage, 38% muscle mass, average bone density"

In [ ]:
query = {"subject":subject,
         "moments":[
            {"time":"0 min", "description":"the subject is outside, it is using a thick blouse"},
            {"time":"1 min", "description":"the subject enters a house, it is using a thick blouse"},
            {"time":"2 min", "description":"the subject turns on a heater at 22 ºC"},
            {"time":"5 min", "description":"no action"},
            {"time":"10 min", "description":"the subject removes it's blouse"}
         ]
         }

In [ ]:
result = test_tool(query)

In [ ]:
result

{'temperatures': [{'time': '0 min', 'temperature': '15.5°C'},
  {'time': '1 min', 'temperature': '18.2°C'},
  {'time': '2 min', 'temperature': '20.1°C'},
  {'time': '5 min', 'temperature': '21.5°C'},
  {'time': '10 min', 'temperature': '23.8°C'}]}

In [ ]:
class TestTool2(Tool, GroqInterface):


    _system_message = '''You are a smarthphone light sensor, carried by a subject, that outputs in JSON. 
The JSON object must use the schema: {'light intensities':[{'time':'str', 'reasoning':'str', 'ligth':'str'}, {'time':'str', 'reasoning':'str', 'ligth':'str'}]}, 
where light must be in lux, and reasoning is a explanation about why this value this moment.

Please use a valid JSON format.'''

    _base_prompt = '''Generate a estimated light intensity for each time, considering the perceptual information from our subject:

Subject: {subject}

Moments: 
{moments}
'''

    def __init__(self, model: Optional[str] = None, api_key: Optional[str] = None):
      
        super().__init__(model, api_key, True, self._system_message)

    def __call__(self, query:Dict[str, str], context:str=None) -> Dict[str, List[str]]:

        prompt = self._base_prompt.format(**query)


        return json.loads(GroqInterface.__call__(self, prompt=prompt))

In [ ]:
test_tool2 = TestTool2()

In [ ]:
result2 = test_tool2(query)

In [ ]:
result2

{'light intensities': [{'time': '0 min',
   'reasoning': 'subject is outside on a normal day',
   'light': '10000 lux'},
  {'time': '1 min',
   'reasoning': 'subject enters a house, but still has daylight through windows',
   'light': '500 lux'},
  {'time': '2 min',
   'reasoning': 'subject turns on a heater, but no significant change in light',
   'light': '500 lux'},
  {'time': '5 min',
   'reasoning': 'no action, light remains the same',
   'light': '500 lux'},
  {'time': '10 min',
   'reasoning': 'subject removes blouse, but light does not change',
   'light': '500 lux'}]}